# "[OptimizationTheory] CH02. Search based Optimization"
> Optimization theory summary note.

- toc: false
- badges: false
- comments: false
- categories: [optimization-theory]
- hide_{github,colab,binder,deepnote}_badge: true

#### 2.1. Grid Search

$$
\mathbf{x}^* = \underset{\mathbf{x}}{\mathrm{argmin}} \,\ f(\mathbf{x}) \,\ \text{where} \,\ f:(0,1)^n \rightarrow \mathbb{R} \,\ \text{and} \,\ \mathbf{x} \in \{(a_1, a_2, \cdots, a_n) : a_1, a_2, \cdots, a_n \in \{0, \frac{1}{m}, \frac{2}{m}, \cdots, \frac{m - 1}{m}\} \}, \,\ m \in \mathbb{N}
$$

__Example)__<br>
- In KNN classification, we can obtain $k$ with grid search.
- In SVM classification with RBF kernel, we can obtain $c$ and $\gamma$.

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm, datasets

iris = datasets.load_iris()
params = {
    'kernel': ('linear', 'rbf'),
    'C': [1, 10]
}

svc = svm.SVC()
clf = GridSearchCV(svc, params)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00038528, 0.00042386, 0.00036063, 0.00038018]),
 'std_fit_time': array([4.15368686e-05, 6.48989211e-06, 1.46394444e-05, 7.80994636e-06]),
 'mean_score_time': array([0.00018463, 0.00020938, 0.00017271, 0.00018597]),
 'std_score_time': array([1.26002358e-05, 1.30935003e-06, 6.32595976e-07, 3.16657214e-06]),
 'param_C': masked_array(data=[1, 1, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'}],
 'split0_test_score': array([0.96666667, 0.96666667, 1.        , 0.96666667]),
 'split1_test_score': array([1.        , 0.96666667, 1.        , 1.        ]),
 'split2_test_score': array([0.96666667, 0.96666667, 0

In [2]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

#### 2.2. Random Search

Random search randomly selects a point to search for. It is generally faster than grid search.

In [3]:
from sklearn import svm, datasets
from sklearn.model_selection import RandomizedSearchCV

iris = datasets.load_iris()
params = {
    'kernel': ('linear', 'rbf'),
    'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

svc = svm.SVC()
clf = RandomizedSearchCV(svc, params, n_iter=5)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.0004076 , 0.00036163, 0.00039191, 0.00040193, 0.00035834]),
 'std_fit_time': array([2.62769725e-05, 5.79195235e-06, 4.55473096e-06, 1.88120569e-05,
        1.21943587e-05]),
 'mean_score_time': array([0.00020247, 0.00017519, 0.00019407, 0.00019431, 0.00019393]),
 'std_score_time': array([1.38532479e-05, 6.97552626e-07, 2.92001932e-06, 2.64633883e-06,
        3.75817868e-05]),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'rbf', 'linear'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[8, 4, 4, 5, 6],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'kernel': 'rbf', 'C': 8},
  {'kernel': 'linear', 'C': 4},
  {'kernel': 'rbf', 'C': 4},
  {'kernel': 'rbf', 'C': 5},
  {'kernel': 'linear', 'C': 6}],
 'split0_test_score': array([0.96666667, 0.96666667, 0.96666667, 0.96666667, 1.        ]),


In [4]:
clf.best_params_

{'kernel': 'rbf', 'C': 8}

#### 2.3. Baysian Optimization
Bayesian optimization is a sequential design strategy for global optimization of black-box functions that does not assume any functional forms. The above methods assume that the results of each trial are independent of each other. However, bayesian optimization optimizes by selecting the next candidate point using the results of each trial.

In [5]:
def black_box_function(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1

In [6]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'x': (2, 4), 'y': (-3, 3)}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=1,
)

In [7]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -7.135    |  2.834    |  1.322    |
|  2        | -7.78     |  2.0      | -1.186    |
|  3        | -7.11     |  2.218    | -0.7867   |
|  4        | -12.4     |  3.66     |  0.9608   |
|  5        | -6.999    |  2.23     | -0.7392   |


In [8]:
optimizer.res

[{'target': -7.135455292718879,
  'params': {'x': 2.8340440094051482, 'y': 1.3219469606529488}},
 {'target': -7.779531005607566,
  'params': {'x': 2.0002287496346898, 'y': -1.1860045642089614}},
 {'target': -7.109925819441113,
  'params': {'x': 2.2175526295255183, 'y': -0.7867249801593896}},
 {'target': -12.397162416009818,
  'params': {'x': 3.660003815774634, 'y': 0.9608275029525108}},
 {'target': -6.999472814518675,
  'params': {'x': 2.2303920156083024, 'y': -0.7392021938893159}}]

In [9]:
optimizer.max

{'target': -6.999472814518675,
 'params': {'x': 2.2303920156083024, 'y': -0.7392021938893159}}

#### 2.4. Golden-Section Search

Golden-section search algorithm is search algorithm for finding a minumum on an interval $[x_l, x_u]$ with a single minimum(unimodal interval). It uses the golden ratio $\phi = 1.6180 \cdots $ to determine location of two interior points $x_1$ and $x_2$. By using the golden ratio, one of the interior points can be re-used in the next iteration.

$$
\begin{matrix}
\text{Let} \,\ d = (\phi - 1)(x_u - x_l) \\
x_1 = x_l + d, \,\ x_2 = x_u - d
\end{matrix}
$$

Similarily, compute new $d$ about $x_1$ and $x_2$. Afterwards, it repeats the specified number of times or until the relative error is lower than the specified threshold.

In [11]:
from scipy import optimize

def f(x):
    return (x - 1)**2

minimum = optimize.golden(f, brack=(0, 5))
minimum

1.000000003917054